In [1]:
from datetime import datetime, timedelta #Modulos del paquete datetime, que me permiten manipular fechas
import numpy as np  #libreria de matematicas
import pandas as pd #libreria para manejar tablas lee en formato txt,  
import os #libreria para tener comandos del sistema


In [2]:
I0 = 1367.
meridiano_ref = -75.
lat = 4.06
lon = -73.47
def declinacion_solar( dia_juliano ):
    dia_rad=2.*np.pi*(dia_juliano-1.)/365.
    return 0.006918-0.399912*np.cos(dia_rad) + 0.070257*np.sin(dia_rad) - 0.006758*np.cos(2.*dia_rad) + 0.00097*np.sin(2.*dia_rad) - 0.002697*np.cos(3.*dia_rad)

def e_t( dia_juliano ):
    dia_rad=2.*np.pi*(dia_juliano-1.)/365.
    return (0.000075 + 0.001868*np.cos(dia_rad)-0.032077*np.sin(dia_rad)-0.014615*np.cos(2.*dia_rad)-0.04089*np.sin(2.*dia_rad))*229.18

def tiempo_solar_verdadero( dia_juliano, lon, hlc ):
    return ((lon-meridiano_ref)*4. + e_t( dia_juliano ))/60. + hlc

def distancia( dia_juliano):
    dia_rad=2.*np.pi*(dia_juliano-1.)/365.
    return 1.00011+0.034221*np.cos(dia_rad) + 0.00128*np.sin(dia_rad)+0.000719*np.cos(2.*dia_rad) + 0.000077*np.sin(2.*dia_rad) 

def cos_zenit( lat, lon, dia_juliano, hlc ):
    lat = lat*np.pi/180.
    decl = declinacion_solar(dia_juliano) 
    angulo_horario = (12 - tiempo_solar_verdadero( dia_juliano, lon, hlc ))*15.*np.pi/180.
    return np.sin(lat)*np.sin(decl) + np.cos(lat)*np.cos(decl)*np.cos(angulo_horario)

In [3]:
os.chdir('/home/dario/Escritorio')#codigo para pararme en una carpeta o lugar
Estacion = pd.read_csv('Estacion/estacionEnero.csv')
Estacion = Estacion.set_index('Fecha')
Estacion.index = pd.to_datetime(Estacion.index, dayfirst=True)
Nivel = pd.read_csv('Rapido/Eneroprueba.csv')
Fechas = [datetime.strptime(Nivel['fecha'][i] + Nivel['hora '][i], '%d/%m/%y%H:%M:%S') for i in range(Nivel.shape[0])]
Nivel['FECHA'] = Fechas
Tabla = pd.read_csv('Tabla 3.csv')
Tabla = Tabla.set_index('Hora')
Tabla.loc[Nivel['hora '][0]]
Tablaco=pd.read_csv('Tablaco.csv')

IOError: [Errno 2] File Tabla 3.csv does not exist: 'Tabla 3.csv'

In [161]:
Dis_Solar = [distancia(int(i.strftime('%j'))) for i in Nivel['FECHA']]
Cos_Zenit = [cos_zenit(lat=lat, lon=lon, dia_juliano=int(i.strftime('%j')), 
             hlc=(int(i.strftime('%-H')) + float(i.strftime('%-M'))/60)) for i in Nivel['FECHA']] 
Nivel['DisSolar'] = Dis_Solar
Nivel['CosZenit'] = Cos_Zenit

Nivel['Vmax'] = [Tabla.loc[i]['Vmax'] for i in Nivel['hora ']]
Nivel['Vmin'] = [Tabla.loc[i]['Vmin'] for i in Nivel['hora ']]




In [162]:
Nivel['Ref']= (Nivel['Ndigital']-29)*1.366*0.00116
Nivel['RMax']= (Nivel['Vmax'])*1.366*0.00116
Nivel['Rmin']= (Nivel['Vmin'])*1.366*0.00116
Nivel['Rp-Rmin']=(Nivel['Ref']-Nivel['Rmin'])
Nivel['RMax-Rmin']=(Nivel['RMax']-Nivel['Rmin'])
Nivel['c']=(Nivel['Rp-Rmin']/Nivel['RMax-Rmin'])
Nivel.loc[Nivel['c'] > 1, 'c'] = 1
Nivel.loc[Nivel['c'] <= 0, 'c'] = np.nan
Nivel['cprom'] = [1 - Nivel.loc[[i.strftime('%Y%m%d') == j.strftime('%Y%m%d') for i in Nivel['FECHA']], :].mean()['c'] for j in Nivel['FECHA']]


In [163]:
Nivel[['FECHA','Ndigital','Ref','RMax', 'Rmin','DisSolar','CosZenit','Rp-Rmin','RMax-Rmin','c', 'cprom']]

,FECHA,Ndigital,Ref,RMax,Rmin,DisSolar,CosZenit,Rp-Rmin,RMax-Rmin,c,cprom
0,2016-05-01 06:45:00,98,0.109335,0.421414,0.119000,0.984017,0.244354,-0.009666,0.302413,NaN,0.665292
1,2016-05-01 09:45:00,321,0.462692,0.758133,0.237842,0.984017,0.840074,0.224849,0.520290,0.432161,0.665292
2,2016-05-01 12:45:00,320,0.461107,0.706951,0.219778,0.984017,0.954517,0.241328,0.487173,0.495365,0.665292
3,2016-05-01 13:15:00,109,0.126765,0.624792,0.190781,0.984017,0.917027,-0.064016,0.434011,NaN,0.665292
4,2016-05-01 13:45:00,163,0.212331,0.667892,0.205993,0.984017,0.864163,0.006338,0.461899,0.013722,0.665292
5,2016-05-01 14:15:00,88,0.093489,0.706951,0.219778,0.984017,0.796829,-0.126289,0.487173,NaN,0.665292
6,2016-05-01 14:45:00,69,0.063382,0.612670,0.186503,0.984017,0.716177,-0.123120,0.426167,NaN,0.665292
7,2016-05-01 15:15:00,177,0.234515,0.416026,0.117099,0.984017,0.623587,0.117416,0.298927,0.392791,0.665292
8,2016-05-01 15:45:00,49,0.031691,0.430842,0.122328,0.984017,0.520644,-0.090637,0.308514,NaN,0.665292
9,2016-05-01 16:15:00,57,0.044368,0.284032,0.070513,0.984017,0.409109,-0.026145,0.213519,NaN,0.665292


In [164]:
Nivel[['FECHA','cprom']]
FECHAS = []
dia  = Nivel['FECHA'][0]
FECHAS = [datetime(dia.year, dia.month, dia.day)]
cpromLista = [Nivel['cprom'][0]]
for i in range(len(Nivel['FECHA'])):
    if Nivel['FECHA'][i].day != dia.day:
        cpromLista.append(Nivel['cprom'][i])
        dia = Nivel['FECHA'][i]
        FECHAS.append(datetime(dia.year, dia.month, dia.day))
df_final = pd.DataFrame({'FECHA':FECHAS, 'CPROM':cpromLista})
estacionLista = []
for i in df_final['FECHA']:
    estacionLista.append(Estacion.loc[Estacion.index == i]['Estacion'][0])
    
df_final['Estacion'] = estacionLista
df_final['J'] = [int(i.strftime('%j')) for i in df_final['FECHA']]
df_final = df_final.set_index('FECHA')
df_final.to_csv('daeio4.csv')